In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import time
import altair as alt
from datetime import datetime
import warnings
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import altair as alt
import geopandas as gpd
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, HoverTool
from bokeh.palettes import brewer
import json

import nedu_prediction as nedu

# show all columns in the dataframe
pd.set_option('max_columns', None)

warnings.filterwarnings("ignore")

# show all columns in the dataframe
pd.set_option('max_columns', None)

In [ ]:
orig_dir = os.getcwd()

## Spoor 2
1. Lees kleinverbruik bestand in (kleinverbruikgegevens_data.h5)
2. Voeg PROFIEL kolom toe obv soort aansluiting  
    * E1 profiel : 1x25, 3x25, 1x20 (met grote en kleine X in bestand), rest is E2
    * Obv laag tarief percentage < 50%, profiel A anders B
    * Als profiel E1 en B en postcodegebied (eerste 2 cijfers postcode) < 65 (zuiden) dan profiel C  
    Bijv:  
    SOORT_AANSLUITING = 1X25, SJV_LAAG_TARIEF_PERC = 60 en POSTCODE_AREA = 56 dan E1A  
    SOORT_AANSLUITING = 1x40, SJV_LAAG_TARIEF_PERC = 60 en POSTCODE_AREA = 89 dan E2B  
    SOORT_AANSLUITING = 3x25, SJV_LAAG_TARIEF_PERC = 40 en POSTCODE_AREA = 55 dan E1C  
3. Verdeel in train en test set op split datum '2018-01-01'
4. Creëer model en roep de fit functie aan op de train set
5. Bepaal de voorspelling op de test set en bereken de nauwkeurigheid (MAPE, RME, R2, )
6. Bepaal de voorspelling voor 2021-2023
7. Visualiseer de voorspellingen in 1 plot met meerdere kleuren mbv altair
8. Visualiseer de teruglevering in 1 plot met meerdere kleuren mbv altair

# Lees predictie functies

In [ ]:
# Zorg er voor dat de veranderingen in verbruik_predictie ook meegenomen worden
from imp import reload
import verbruik_predictie as vp
reload(vp)


# Lees kleinverbruik data

In [ ]:
# variables used in script
data_processed_location = '../data/processed'

if 'processed' not in os.getcwd():
    os.chdir(data_processed_location)

In [ ]:
# kleinverbruikgegevens gegevens inlezen
df = pd.read_hdf('kleinverbruikgegevens_data.h5')

#Delete 2021 data by keeping JAAR < 2021
df = df[df['JAAR'] < 2021]

# Voeg Features toe
Voeg profiel toe op basis van SOORT_AANSLUITING en de SJV_TOTAAL per profiel

In [ ]:
# Spoor 2. Voeg profiel toe aan verbruiksgegevens. SOORT_AANSLUITING bepaald het profiel
# Omdat de E2 profielen in de NEDU set niet consistent zijn (verschil tussen 2010-2017 en 2018-), gebruiken we deze niet
profiel_E1 = ['1X25','3X25', '1X20', '1x25', '3x25', '1x20'] # de rest is profiel E2

# Voeg in de kleinverbruikgegevens, het bijpassende profiel voor de soort aansluiting toe
def vervang_door_profiel(aansluiting, postcode, percentage):
    # Bepaal basisprofiel
    profiel = 'E1' # if aansluiting in profiel_E1 else 'E2' # Gebruiken als we wel een goed E2 profiel hebben

    # Een laag tarief percentage lager dan 50% zetten we in een A profiel
    lh_profiel = 'A' if percentage < 50 else 'B'

    # Bepaal welk laag tarief gebied de postcode zit
    if profiel == 'E1' and lh_profiel == 'B':
        postcode_area = int(postcode[:2]) # PC4
        # Postcodes < 65 is Noord-Brabant of Limburg
        if postcode_area < 65:
            lh_profiel = 'C'
    return profiel + lh_profiel

df["PROFIEL"] = np.vectorize(vervang_door_profiel)(df.SOORT_AANSLUITING, df.POSTCODE_VAN, df.SJV_LAAG_TARIEF_PERC)
df.PC4 = df.PC4.astype('int')
print (f'#E1A = {df[df.PROFIEL == "E1A"].PROFIEL.count()}')
print (f'#E1B = {df[df.PROFIEL == "E1B"].PROFIEL.count()}')
print (f'#E1C = {df[df.PROFIEL == "E1C"].PROFIEL.count()}')

In [ ]:
# Voeg features toe per PC6
df["E1A_TOTAAL"] = df[df.PROFIEL == 'E1A'].SJV_TOTAAL
df["E1B_TOTAAL"] = df[df.PROFIEL == 'E1B'].SJV_TOTAAL
df["E1C_TOTAAL"] = df[df.PROFIEL == 'E1C'].SJV_TOTAAL
df["WEIGHTED_LEVERINGSRICHTING_PERC"] = df.AANSLUITINGEN_AANTAL * df.LEVERINGSRICHTING_PERC

# Rol op tot PC4

In [ ]:
# Rol op tot PC4
df_verbruik = df.groupby(['PC4','JAAR']).agg({'SJV_TOTAAL':'sum', 'E1A_TOTAAL' : 'sum', 'E1B_TOTAAL' : 'sum', 'E1C_TOTAAL': 'sum', 'AANSLUITINGEN_AANTAL':'sum', 'WEIGHTED_LEVERINGSRICHTING_PERC': 'sum'})
df_verbruik['LEVERINGSRICHTING_PERC'] = df_verbruik['WEIGHTED_LEVERINGSRICHTING_PERC'] / df_verbruik['AANSLUITINGEN_AANTAL']
df_verbruik = df_verbruik.drop(columns=['WEIGHTED_LEVERINGSRICHTING_PERC'])
df_verbruik

In [ ]:
# Verhuis de multi-level index naar kolommen en maak de index een simpele range van getallen
df_verbruik['PC4'] = df_verbruik.index.get_level_values('PC4')
df_verbruik['JAAR'] = df_verbruik.index.get_level_values('JAAR')
df_verbruik.index = range(len(df_verbruik))
df_verbruik.SJV_TOTAAL = df_verbruik.SJV_TOTAAL.astype('int')
df_verbruik.E1A_TOTAAL = df_verbruik.E1A_TOTAAL.astype('int')
df_verbruik.E1B_TOTAAL = df_verbruik.E1B_TOTAAL.astype('int')
df_verbruik.E1C_TOTAAL = df_verbruik.E1C_TOTAAL.astype('int')
df_verbruik

# Voeg predictie toe voor de jaren 2021 tot 2023

In [ ]:
# Voorspel het verbruik met lineaire regressie
df_pred = vp.predict_verbruik_lr(df_verbruik, predict_type='mid')
df_pred[df_pred.PC4 == 4251]

In [ ]:
accuracy = vp.calc_accuracy_lr(df_verbruik)
accuracy

In [ ]:
# Check de merge voor postcode 4251
df_verbruik_en_pred = pd.merge(df_verbruik, df_pred, how='outer')
df_verbruik_en_pred[df_verbruik_en_pred.PC4 == 4251].head(14)

# NEDU profielen

## Spoor 1
1. Lees NEDU profielen bestand in (nedu_files.h5)
2. Verwijder alle niet gebruikte profielen (E3, E4) en jaar 2021
3. Rol de kwartierdata op tot data per dag (optellen alle profielfactoren per kwartier)
4. Transformeer het dataframe om samenvoegen met verbruikdata mogelijk te maken op profiel:
    - Oud: (datum, E1A, E1B, E1C, E2A, E2B), bijv.:
        2010-01-01  0.003448  0.003514  0.003620  0.002332  0.003061
    - Nieuw: (datum, PROFIEL, FACTOR), bijv.:   
        2010-01-01  E1A  0.003448  
        2010-01-01  E1B  0.003514  
        2010-01-01  E1C  0.003620  
        2010-01-01  E2A  0.002332  
        2010-01-01  E2B  0.003061  
5. Vermenigvuldig de FACTOR kolom met 1e5 (stabiliteit model wordt dan beter)
5. Verdeel in train en test set op split datum '2018-01-01'
6. Creëer model en roep de fit functie aan op de train set
7. Bepaal de voorspelling op de test set en bereken de nauwkeurigheid (MAPE, RME, R2, )
8. Bepaal de voorspelling voor 2021-2023
9. Visualiseer de voorspellingen in 1 plot met meerdere kleuren mbv altair

In [ ]:
# Lees NEDU profielen bestand in (nedu_files.h5)
# variables used in script
data_processed_location = '../data/processed'

if 'processed' not in os.getcwd():
    os.chdir(data_processed_location)

# NEDU profielen
df_nedu_profielen = pd.read_hdf('nedu_files.h5')
df_nedu_profielen_origineel = df_nedu_profielen

# mapping van PC4 buurt naar RES regio
df_pc4_res = pd.read_hdf('pc4_res.h5')

In [ ]:
# Verwijder alle niet gebruikte profielen (E3, E4) 
drop_onnodige_profielen = {"E3A","E3B", 'E3C', 'E3D', 'E4A'}
df_nedu_profielen = df_nedu_profielen.drop(columns = drop_onnodige_profielen)
# Verwijder jaar 2021
df_nedu_profielen = df_nedu_profielen[df_nedu_profielen.DatumTijd < '2021-01-01']

In [ ]:
# Rol de kwartierdata op tot data per dag (optellen alle profielfactoren per kwartier)
# feautures toevoegen zodat functie groupby werkt: jaar, maand, dag
df_nedu_profielen.info()

In [ ]:
df_nedu_profielen['jaar'] = df_nedu_profielen.DatumTijd.dt.year
df_nedu_profielen['maand'] = df_nedu_profielen.DatumTijd.dt.month
df_nedu_profielen['dag'] = df_nedu_profielen.DatumTijd.dt.day

df_nedu_profielen = df_nedu_profielen.groupby(['jaar','maand','dag']).agg({'E1A':'sum', 'E1B':'sum', 'E1C':'sum', 'E2A':'sum', 'E2B':'sum'})

def maak_datum(jaar,maand,dag):
    return format(jaar,'04d') + '-' + format(maand,'02d') + '-' + format(dag,'02d')
df_nedu_profielen.index = np.vectorize(maak_datum)(df_nedu_profielen.index.get_level_values('jaar'), df_nedu_profielen.index.get_level_values('maand'), df_nedu_profielen.index.get_level_values('dag'))

In [ ]:
# Verschillende dataframes maken voor de verschillende profielen
# op basis van df_nedu_profielen dataframe
df_nedu_e1a = pd.DataFrame(df_nedu_profielen['E1A']).reset_index().rename(columns={"index":"DATUM","E1A": "VERBRUIKS_FACTOR"})
df_nedu_e1b = pd.DataFrame(df_nedu_profielen['E1B']).reset_index().rename(columns={"index":"DATUM","E1B": "VERBRUIKS_FACTOR"})
df_nedu_e1c = pd.DataFrame(df_nedu_profielen['E1C']).reset_index().rename(columns={"index":"DATUM","E1C": "VERBRUIKS_FACTOR"})
df_nedu_e2a = pd.DataFrame(df_nedu_profielen['E2A']).reset_index().rename(columns={"index":"DATUM","E2A": "VERBRUIKS_FACTOR"})
df_nedu_e2b = pd.DataFrame(df_nedu_profielen['E2B']).reset_index().rename(columns={"index":"DATUM","E2B": "VERBRUIKS_FACTOR"})

# Vermenigvuldig de FACTOR kolom met 1e5 (stabiliteit model wordt dan beter)
df_nedu_e1a['VERBRUIKS_FACTOR'] = df_nedu_e1a['VERBRUIKS_FACTOR'] * 1e5
df_nedu_e1b['VERBRUIKS_FACTOR'] = df_nedu_e1b['VERBRUIKS_FACTOR'] * 1e5
df_nedu_e1c['VERBRUIKS_FACTOR'] = df_nedu_e1c['VERBRUIKS_FACTOR'] * 1e5
df_nedu_e2a['VERBRUIKS_FACTOR'] = df_nedu_e2a['VERBRUIKS_FACTOR'] * 1e5
df_nedu_e2b['VERBRUIKS_FACTOR'] = df_nedu_e2b['VERBRUIKS_FACTOR'] * 1e5

In [ ]:
# Prophet
# train model per profiel voor bepaalde train periode
from_date = '2010-01-01'
split_date = '2019-01-01'

nedu.train_model_nedu_profielen_prophet(df_nedu_e1a, profile='e1a', from_date=from_date, split_date=split_date)
nedu.train_model_nedu_profielen_prophet(df_nedu_e1b, profile='e1b', from_date=from_date, split_date=split_date)
nedu.train_model_nedu_profielen_prophet(df_nedu_e1c, profile='e1c', from_date=from_date, split_date=split_date)
nedu.train_model_nedu_profielen_prophet(df_nedu_e2a, profile='e2a', from_date=from_date, split_date=split_date)
nedu.train_model_nedu_profielen_prophet(df_nedu_e2b, profile='e2b', from_date=from_date, split_date=split_date)

In [ ]:
# maak een LOW prediction voor een profiel voor een bepaalde periode
predict_start = '2021-01-01'
predict_end   = '2023-12-31'
predict_type  = 'low'
df_nedu_e1a_low_pred = nedu.predict_nedu_profielen_prophet(model='NEDU_PRO_e1a_'+split_date+'.pkl', start=predict_start, end=predict_end, predict_type=predict_type)
df_nedu_e1b_low_pred = nedu.predict_nedu_profielen_prophet(model='NEDU_PRO_e1b_'+split_date+'.pkl', start=predict_start, end=predict_end, predict_type=predict_type)
df_nedu_e1c_low_pred = nedu.predict_nedu_profielen_prophet(model='NEDU_PRO_e1c_'+split_date+'.pkl', start=predict_start, end=predict_end, predict_type=predict_type)
df_nedu_e2a_low_pred = nedu.predict_nedu_profielen_prophet(model='NEDU_PRO_e2a_'+split_date+'.pkl', start=predict_start, end=predict_end, predict_type=predict_type)
df_nedu_e2b_low_pred = nedu.predict_nedu_profielen_prophet(model='NEDU_PRO_e2b_'+split_date+'.pkl', start=predict_start, end=predict_end, predict_type=predict_type)

In [ ]:
# maak een MID prediction voor een profiel voor een bepaalde periode
predict_start = '2021-01-01'
predict_end   = '2023-12-31'
predict_type  = 'mid'
df_nedu_e1a_mid_pred = nedu.predict_nedu_profielen_prophet(model='NEDU_PRO_e1a_'+split_date+'.pkl', start=predict_start, end=predict_end, predict_type=predict_type)
df_nedu_e1b_mid_pred = nedu.predict_nedu_profielen_prophet(model='NEDU_PRO_e1b_'+split_date+'.pkl', start=predict_start, end=predict_end, predict_type=predict_type)
df_nedu_e1c_mid_pred = nedu.predict_nedu_profielen_prophet(model='NEDU_PRO_e1c_'+split_date+'.pkl', start=predict_start, end=predict_end, predict_type=predict_type)
df_nedu_e2a_mid_pred = nedu.predict_nedu_profielen_prophet(model='NEDU_PRO_e2a_'+split_date+'.pkl', start=predict_start, end=predict_end, predict_type=predict_type)
df_nedu_e2b_mid_pred = nedu.predict_nedu_profielen_prophet(model='NEDU_PRO_e2b_'+split_date+'.pkl', start=predict_start, end=predict_end, predict_type=predict_type)

In [ ]:
# maak een HIGH prediction voor een profiel voor een bepaalde periode
predict_start = '2021-01-01'
predict_end   = '2023-12-31'
predict_type  = 'high'
df_nedu_e1a_high_pred = nedu.predict_nedu_profielen_prophet(model='NEDU_PRO_e1a_'+split_date+'.pkl', start=predict_start, end=predict_end, predict_type=predict_type)
df_nedu_e1b_high_pred = nedu.predict_nedu_profielen_prophet(model='NEDU_PRO_e1b_'+split_date+'.pkl', start=predict_start, end=predict_end, predict_type=predict_type)
df_nedu_e1c_high_pred = nedu.predict_nedu_profielen_prophet(model='NEDU_PRO_e1c_'+split_date+'.pkl', start=predict_start, end=predict_end, predict_type=predict_type)
df_nedu_e2a_high_pred = nedu.predict_nedu_profielen_prophet(model='NEDU_PRO_e2a_'+split_date+'.pkl', start=predict_start, end=predict_end, predict_type=predict_type)
df_nedu_e2b_high_pred = nedu.predict_nedu_profielen_prophet(model='NEDU_PRO_e2b_'+split_date+'.pkl', start=predict_start, end=predict_end, predict_type=predict_type)

In [ ]:
# DATUM aanpassen naar datetime formaat
df_nedu_e1a['DATUM'] = pd.to_datetime(df_nedu_e1a['DATUM'])
df_nedu_e1b['DATUM'] = pd.to_datetime(df_nedu_e1b['DATUM'])
df_nedu_e1c['DATUM'] = pd.to_datetime(df_nedu_e1c['DATUM'])
df_nedu_e2a['DATUM'] = pd.to_datetime(df_nedu_e2a['DATUM'])
df_nedu_e2b['DATUM'] = pd.to_datetime(df_nedu_e2b['DATUM'])

In [ ]:
# Interactive zoom on both X and Y axis with two lower graphs.
df_nedu = df_nedu_e1a
df_nedu_pred_low = df_nedu_e1a_low_pred
df_nedu_pred_mid = df_nedu_e1a_mid_pred
df_nedu_pred_high = df_nedu_e1a_high_pred

interval = alt.selection_interval(encodings=['x'])
verbruik_range = alt.selection_interval(encodings=['y'])

base = alt.Chart(df_nedu.reset_index()).mark_line(clip=True).encode(
    x='DATUM:T',
    y='VERBRUIKS_FACTOR:Q'
)

forecast_low = alt.Chart(df_nedu_pred_low.reset_index()).mark_line(clip=True).encode(
                     x='DATUM:T',
                     y='VERBRUIKS_FACTOR:Q'
                     )
    
forecast_mid = alt.Chart(df_nedu_pred_mid.reset_index()).mark_line(clip=True).encode(
                     x='DATUM:T',
                     y='VERBRUIKS_FACTOR:Q'
                     )

forecast_high = alt.Chart(df_nedu_pred_high.reset_index()).mark_line(clip=True).encode(
                     x='DATUM:T',
                     y='VERBRUIKS_FACTOR:Q'
                     )
    
chart = base.encode(
    x=alt.X('DATUM:T', scale=alt.Scale(domain=interval.ref())),
    y=alt.Y('VERBRUIKS_FACTOR:Q', scale=alt.Scale(domain=verbruik_range.ref()))
).properties(
    width=800,
    height=400)

forecast_low_chart = forecast_low.encode(
    x=alt.X('DATUM:T', scale=alt.Scale(domain=interval.ref())),
    y=alt.Y('VERBRUIKS_FACTOR:Q', scale=alt.Scale(domain=verbruik_range.ref())),color=alt.value('orange')
).properties(
    width=800,
    height=400)

forecast_mid_chart = forecast_mid.encode(
    x=alt.X('DATUM:T', scale=alt.Scale(domain=interval.ref())),
    y=alt.Y('VERBRUIKS_FACTOR:Q', scale=alt.Scale(domain=verbruik_range.ref())),color=alt.value('red')
).properties(
    width=800,
    height=400)

forecast_high_chart = forecast_high.encode(
    x=alt.X('DATUM:T', scale=alt.Scale(domain=interval.ref())),
    y=alt.Y('VERBRUIKS_FACTOR:Q', scale=alt.Scale(domain=verbruik_range.ref())),color=alt.value('orange')
).properties(
    width=800,
    height=400)
    
view = base+forecast_low.add_selection(
    interval
).properties(
    width=800,
    height=50,
)

window = base+forecast_low.add_selection(
    verbruik_range
).properties(
    width=800,
    height=100,
)

chart + forecast_low_chart + forecast_mid_chart + forecast_high_chart & view & window

In [ ]:
# MID prediction dataframes samenvoegen met bestaande NEDU dataframes
df_nedu_e1a_all = pd.concat([df_nedu_e1a[df_nedu_e1a['DATUM'] < predict_start], df_nedu_e1a_mid_pred])
df_nedu_e1b_all = pd.concat([df_nedu_e1b[df_nedu_e1b['DATUM'] < predict_start], df_nedu_e1b_mid_pred])
df_nedu_e1c_all = pd.concat([df_nedu_e1c[df_nedu_e1c['DATUM'] < predict_start], df_nedu_e1c_mid_pred])
df_nedu_e2a_all = pd.concat([df_nedu_e2a[df_nedu_e2a['DATUM'] < predict_start], df_nedu_e2a_mid_pred])
df_nedu_e2b_all = pd.concat([df_nedu_e2b[df_nedu_e2b['DATUM'] < predict_start], df_nedu_e2b_mid_pred])

In [ ]:
# verbruiksfactor weer delen door 1e5
df_nedu_e1a_all['VERBRUIKS_FACTOR'] = df_nedu_e1a_all['VERBRUIKS_FACTOR'] / 1e5
df_nedu_e1b_all['VERBRUIKS_FACTOR'] = df_nedu_e1b_all['VERBRUIKS_FACTOR'] / 1e5
df_nedu_e1c_all['VERBRUIKS_FACTOR'] = df_nedu_e1c_all['VERBRUIKS_FACTOR'] / 1e5
df_nedu_e2a_all['VERBRUIKS_FACTOR'] = df_nedu_e2a_all['VERBRUIKS_FACTOR'] / 1e5
df_nedu_e2b_all['VERBRUIKS_FACTOR'] = df_nedu_e2b_all['VERBRUIKS_FACTOR'] / 1e5

In [ ]:
df_nedu_e1a_all2 = df_nedu_e1a_all.rename(columns={'VERBRUIKS_FACTOR': 'E1A'})
df_nedu_e1b_all2 = df_nedu_e1b_all.rename(columns={'VERBRUIKS_FACTOR': 'E1B'})
df_nedu_e1c_all2 = df_nedu_e1c_all.rename(columns={'VERBRUIKS_FACTOR': 'E1C'})
df_nedu_e2a_all2 = df_nedu_e2a_all.rename(columns={'VERBRUIKS_FACTOR': 'E2A'})
df_nedu_e2b_all2 = df_nedu_e2b_all.rename(columns={'VERBRUIKS_FACTOR': 'E2B'})

In [ ]:
# de dataframes samenvoegen
df_nedu_profielen = pd.merge(df_nedu_e1a_all2, df_nedu_e1b_all2, on='DATUM', how='left')
df_nedu_profielen = pd.merge(df_nedu_profielen, df_nedu_e1c_all2, on='DATUM', how='left')
df_nedu_profielen = pd.merge(df_nedu_profielen, df_nedu_e2a_all2, on='DATUM', how='left')
df_nedu_profielen = pd.merge(df_nedu_profielen, df_nedu_e2b_all2, on='DATUM', how='left')

In [ ]:
# jaar toevoegen aan nedu profielen om op te kunnen joinen met jaar verbruiksdata
df_nedu_profielen['JAAR'] = df_nedu_profielen['DATUM'].dt.year

In [ ]:
df_nedu_profielen[df_nedu_profielen['DATUM'] >= '2021-01-01'].head()

# Combinatiespoor
Combineer de dataframes van de NEDU profielen met voorspelling en het dataframe van de kleinverbruikgegevens inclusief voorspelling

1. Voeg de profielen en kleinverbruik samen op de kolom PROFIEL (zorg er voor dat de kolommen DATUM, SJV_TOTAAL, FACTOR er in zitten)
2. Rol op tot PC4 VERBRUIK, waarbij VERBRUIK = SJV_TOTAAL * FACTOR
3. Lees bestand met PC4 en RES gegevens in (pc4_res.h5)
4. Rol op tot RES VERBRUIK
5. Creëer dataframe met de delta van jaar tot jaar per dag per RES

In [ ]:
# Combineer NEDU profielen met de kleinverbruikgegevens
df_combined = pd.merge(df_nedu_profielen, df_verbruik_en_pred, on=['JAAR'], how='left')

In [ ]:
df_combined[df_combined.PC4 == 4251].tail(14)

In [ ]:
# Voeg nieuwe kolom VERBRUIK toe die de som is van het verbruik van de verschillende profielen
df_combined['VERBRUIK'] = df_combined.E1A * df_combined.E1A_TOTAAL + df_combined.E1B * df_combined.E1B_TOTAAL + df_combined.E1C * df_combined.E1C_TOTAAL

# Gooi de tussenkolommen weg. Die hebben we niet meer nodig
df_combined = df_combined.drop(columns=['E1A','E1B','E1C','E1A_TOTAAL','E1B_TOTAAL','E1C_TOTAAL','JAAR'])

In [ ]:
# cross check. Het totaal van een profiel over een heel jaar moet 1 zijn. Dat betekent dat het totaal van de verbruiken gelijk moet zijn aan SJV_TOTAAL
print (f"Totaal verbruik = {df_combined[(df_combined.DATUM < datetime(2011,1,1)) & (df_combined.PC4 == 4251)].agg({'VERBRUIK':'sum'}).values[0]}")
print (f"SJV totaal = {df_combined[(df_combined.DATUM < datetime(2011,1,1)) & (df_combined.PC4 == 4251)].SJV_TOTAAL.values[0]}")

In [ ]:
df_combined[df_combined.PC4 == 4251].tail(14)

# RES Regios toevoegen

In [ ]:
# Change type of PC4 for merge
df_combined['PC4'] = df_combined['PC4'].astype('int64') 

# Merge in RES 
df_combined = pd.merge(df_combined, df_pc4_res, on=['PC4'], how='left')

In [ ]:
df_combined[(df_combined['PC4'] == 4251) & (df_combined['DATUM'] == '2023-01-01')].head()

# Delta Piekverbruik visualiseren

In [ ]:
def bereken_piek_verbruik(df_combined, niveau='PC4'):
    # jaar toevoegen aan nedu profielen om op te kunnen joinen met jaar verbruiksdata
    df_combined['JAAR'] = df_combined['DATUM'].dt.year
    
    # piekverbruik per JAAR
    df_piek = df_combined.groupby([niveau,'JAAR'], as_index=False).agg({'VERBRUIK': 'max'})
    df_piek = df_piek.sort_values([niveau,'JAAR'], ascending=[True, True])
    df_piek = df_piek.rename(columns={'VERBRUIK': 'PIEK_VERBRUIK'})
    
    # piekverbruik van vorige jaar, binnen groupby
    df_piek['PIEK_VERBRUIK_VORIG_JAAR'] = df_piek.groupby(niveau)['PIEK_VERBRUIK'].shift(1)
    # Delta piekverbruik tov vorig jaar
    df_piek['DELTA_VORIG_JAAR'] = df_piek['PIEK_VERBRUIK'] - (df_piek.groupby(niveau)['PIEK_VERBRUIK'].shift(1))
    df_piek['STIJGINGS_PERC'] = (df_piek['DELTA_VORIG_JAAR'] / df_piek['PIEK_VERBRUIK_VORIG_JAAR'] * 100)
    
    return df_piek

In [ ]:
# delta piekverbruik berekenen tov vorige jaar voor PC4 en RES
df_piek_pc4 = bereken_piek_verbruik(df_combined,niveau='PC4')
df_piek_res = bereken_piek_verbruik(df_combined,niveau='RES')

In [ ]:
#df_piek_pc4[df_piek_pc4['PC4'] == 4251]

In [ ]:
#df_piek_res[df_piek_res['RES'] == 'West-Brabant']

In [ ]:
# Polygonen toevoegen
# Lees polygonen bestand in voor PC4
data_raw_pc4_location = '../data/raw/Polygonen/Postcodevlakken_PC_4-shp/'

os.chdir(orig_dir)
if 'Postcodevlakken_PC_4-shp' not in os.getcwd():
    os.chdir(data_raw_pc4_location)
    
file = 'PC4.shp'
pc4 = gpd.read_file(file)

In [ ]:
#mergen met df piek per pc4
pc4['PC4'] = pc4['PC4'].astype('int64') 
df_piek_pc4_geo = pc4.merge(df_piek_pc4, on='PC4', how='left')

In [ ]:
#df_piek_pc4_geo[df_piek_pc4_geo['PC4'] == 4251]

In [ ]:
# plot
jaar = 2023
df_geopandas = df_piek_pc4_geo[df_piek_pc4_geo['JAAR']==jaar]

p = df_geopandas.plot(column='STIJGINGS_PERC', legend=True, figsize = (15,7), cmap='OrRd', vmin=-10, vmax=+10)
p.axis('off')
p.set_title(f'Stijgingspercentage Piekverbruik in {jaar} tov {jaar-1} per PC4')

In [ ]:
#Filter data voor een jaar.
jaar=2020
df_piek_pc4_geo_jaar = df_piek_pc4_geo[df_piek_pc4_geo['JAAR'] == jaar]
merged = df_piek_pc4_geo_jaar[['geometry','JAAR','PC4','STIJGINGS_PERC']]

#Read data to json.
merged_json = json.loads(merged.to_json())
#Convert to String like object.
json_data = json.dumps(merged_json)

#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = json_data)

#Define a sequential multi-hue color palette.
palette = brewer['YlGnBu'][8]

#Reverse color order so that dark blue is highest.
palette = palette[::-1]

#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = -10, high = 10)

#Define custom tick labels for color bar.
tick_labels = {'-10': '<-10%', '-5': '-5%', '0':'0%', '5':'5%', '10':'>10%'}

#Add hover tool
hover = HoverTool(tooltips = [ ('PC4','@PC4'),('% Stijging', '@STIJGINGS_PERC')])

#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 400, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)

#Create figure object.
p = figure(title = f'Stijgingspercentage Piekverbruik per PC4 in {jaar}', plot_height = 600 , plot_width = 450, toolbar_location = None, tools = [hover])
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'STIJGINGS_PERC', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)

#Specify figure layout.
p.add_layout(color_bar, 'below')

#Display figure inline in Jupyter Notebook.
output_notebook()

#Display figure.
show(p)

In [ ]:
# Stijgingspercentage Piekverbruik per RES regio

In [ ]:
# Lees polygonen bestand in voor RES
data_raw_res_location = '../data/raw/Polygonen/Regionale_Energiestrategie_RES_regios-shp/'

os.chdir(orig_dir)
if 'Regionale_Energiestrategie_RES_regios-shp' not in os.getcwd():
    os.chdir(data_raw_res_location)
    
file = '837dfba3-69e2-48a9-9697-892a5c3c4be42020413-1-1z1r3u.mhsfp.shp'
res = gpd.read_file(file).rename(columns={'Regio': 'RES'})

In [ ]:
#mergen met df piek per res
df_piek_res_geo = pd.merge(res, df_piek_res, on='RES', how='left')
df_piek_res_geo = df_piek_res_geo.dropna()

In [ ]:
# plot
jaar = 2023
df_geopandas = df_piek_res_geo[df_piek_res_geo['JAAR']==jaar]

p = df_geopandas.plot(column='STIJGINGS_PERC', legend=True, figsize = (15,6), cmap='OrRd', vmin=-10, vmax=+10)
p.axis('off')
p.set_title(f'Stijgingspercentage in {jaar} tov {jaar-1} per RES')

# Delta van Jaarverbruik visualiseren

In [ ]:
def bereken_jaar_verbruik(df_combined, niveau='PC4'):
    # jaar toevoegen aan nedu profielen om op te kunnen joinen met jaar verbruiksdata
    df_combined['JAAR'] = df_combined['DATUM'].dt.year
    
    # SJV_TOTAAL per JAAR
    df_sjv = df_combined.groupby([niveau,'JAAR'], as_index=False).agg({'SJV_TOTAAL': 'max'})
    df_sjv = df_sjv.sort_values([niveau,'JAAR'], ascending=[True, True])
    #df_sjv = df_sjv.rename(columns={'SJV_TOTAAL': 'SJV_TOTAAL'})
    
    # SJV_TOTAAL van vorige jaar, binnen groupby
    df_sjv['SJV_TOTAAL_VORIG_JAAR'] = df_sjv.groupby(niveau)['SJV_TOTAAL'].shift(1)
    # Delta SJV_TOTAAL tov vorig jaar
    df_sjv['DELTA_VORIG_JAAR'] = df_sjv['SJV_TOTAAL'] - (df_sjv.groupby(niveau)['SJV_TOTAAL'].shift(1))
    df_sjv['STIJGINGS_PERC'] = (df_sjv['DELTA_VORIG_JAAR'] / df_sjv['SJV_TOTAAL_VORIG_JAAR'] * 100)
    
    return df_sjv

In [ ]:
# delta piekverbruik berekenen tov vorige jaar voor PC4 en RES
df_sjv_pc4 = bereken_jaar_verbruik(df_combined,niveau='PC4')
df_sjv_res = bereken_jaar_verbruik(df_combined,niveau='RES')

In [ ]:
#mergen met df sjv per pc4
pc4['PC4'] = pc4['PC4'].astype('int64') 
df_sjv_pc4_geo = pc4.merge(df_sjv_pc4, on='PC4', how='left')

In [ ]:
#df_sjv_pc4_geo[df_sjv_pc4_geo['PC4'] ==  5245]

In [ ]:
jaar = 2021
df_geopandas = df_sjv_pc4_geo[df_sjv_pc4_geo['JAAR']==jaar]

#p = df_geopandas.plot(column='DELTA_VORIG_JAAR', legend=True, figsize = (15,6), cmap='OrRd', vmin=-1000, vmax=1000)
p = df_geopandas.plot(column='STIJGINGS_PERC', legend=True, figsize = (15,12), cmap='PuOr', vmin=-10, vmax=10)
p.axis('off')
p.set_title(f'Stijgingspercentage Jaarverbruik in {jaar} t.o.v. {jaar-1} per PC4')

In [ ]:
#Filter data voor een jaar.
jaar=2020
df_sjv_pc4_geo_jaar = df_sjv_pc4_geo[df_sjv_pc4_geo['JAAR'] == jaar]
merged = df_sjv_pc4_geo_jaar[['geometry','JAAR','PC4','STIJGINGS_PERC']]

#Read data to json.
merged_json = json.loads(merged.to_json())
#Convert to String like object.
json_data = json.dumps(merged_json)

#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = json_data)

#Define a sequential multi-hue color palette.
palette = brewer['YlGnBu'][8]

#Reverse color order so that dark blue is highest.
palette = palette[::-1]

#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = -10, high = 10)

#Define custom tick labels for color bar.
tick_labels = {'-10': '<-10%', '-5': '-5%', '0':'0%', '5':'5%', '10':'>10%'}

#Add hover tool
hover = HoverTool(tooltips = [ ('PC4','@PC4'),('% Stijging', '@STIJGINGS_PERC')])

#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 400, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)

#Create figure object.
p = figure(title = f'Stijgingspercentage Jaarverbuik per PC4 in {jaar}', plot_height = 600 , plot_width = 450, toolbar_location = None, tools = [hover])
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'STIJGINGS_PERC', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)

#Specify figure layout.
p.add_layout(color_bar, 'below')

#Display figure inline in Jupyter Notebook.
output_notebook()

#Display figure.
show(p)

# Leveringsrichting percentage visualiseren

In [ ]:
def bereken_jaar_lev_richting_prec(df_combined, niveau='PC4'):
    # jaar toevoegen aan nedu profielen om op te kunnen joinen met jaar verbruiksdata
    df_combined['JAAR'] = df_combined['DATUM'].dt.year
    
    # LEVERINGSRICHTING_PERC per JAAR
    df_lr = df_combined.groupby([niveau,'JAAR'], as_index=False).agg({'LEVERINGSRICHTING_PERC': 'max'
                                                                     , 'SJV_TOTAAL': 'max'
                                                                     , 'RES': 'max'})
    df_lr = df_lr.sort_values([niveau,'JAAR'], ascending=[True, True])
    
    # LEVERINGSRICHTING_PERC van vorige jaar, binnen groupby
    df_lr['LEVERINGSRICHTING_PERC_VORIG_JAAR'] = df_lr.groupby(niveau)['LEVERINGSRICHTING_PERC'].shift(1)
    # Delta LEVERINGSRICHTING_PERC tov vorig jaar
    df_lr['DELTA_VORIG_JAAR'] = df_lr['LEVERINGSRICHTING_PERC'] - (df_lr.groupby(niveau)['LEVERINGSRICHTING_PERC'].shift(1))
    df_lr['DALINGS_PERC'] = (df_lr['DELTA_VORIG_JAAR'] / df_lr['LEVERINGSRICHTING_PERC_VORIG_JAAR'] * 100 * -1)
    
    return df_lr

In [ ]:
# delta lev richting percentage berekenen tov vorige jaar voor PC4 en RES
df_lr_pc4 = bereken_jaar_lev_richting_prec(df_combined,niveau='PC4')
df_lr_res = bereken_jaar_lev_richting_prec(df_combined,niveau='RES')

In [ ]:
# Look at leveringsrichting as a function of verbruik (proxy for local electricity generation using solar) per year per RES.
# [each dot is a PC4]

alt.data_transformers.disable_max_rows()

select_year = alt.selection_single(
    name='select', fields=['JAAR'], init={'JAAR': 2010},
    bind=alt.binding_range(min=2010, max=2023, step=1))

alt.Chart(df_lr_pc4).mark_point(filled=True).encode(
    alt.X('LEVERINGSRICHTING_PERC', scale=alt.Scale(domain=(0, 110))),
    alt.Y('SJV_TOTAAL', scale=alt.Scale(domain=(0, 60000))),
    alt.Color('RES:N')).add_selection(select_year).transform_filter(select_year).properties(
    width=400,height=400)

In [ ]:
#mergen met df lr per pc4
pc4['PC4'] = pc4['PC4'].astype('int64') 
df_lr_pc4_geo = pc4.merge(df_lr_pc4, on='PC4', how='left')

In [ ]:
#df_lr_pc4_geo[df_lr_pc4_geo['PC4'] == 4822]

In [ ]:
jaar = 2023
df_geopandas = df_lr_pc4_geo[df_lr_pc4_geo['JAAR']==jaar]

p = df_geopandas.plot(column='DALINGS_PERC', legend=True, figsize = (15,12), cmap='PuOr', vmin=-15, vmax=15)
p.axis('off')
p.set_title(f'Dalingspercentage Leveringsrichtingpercentage in {jaar} t.o.v. {jaar-1} per PC4')